# Supervised Learning: Gradient Boosting Regressor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from supervised_learning.cross_validation import PanelDataSplit
from supervised_learning.cross_validation import search_best_model
from sample_panel.merge_datasets import merge_bank_macro_datasets

from supervised_learning.estimate_errors import estimate_median_relative_error
from supervised_learning.estimate_errors import estimate_errors

## Preparing a Data Sample

### Loading data

In [3]:
# Load bank panel data
bank_data = pd.read_csv('df_response_vars.csv')

In [4]:
# Load macroeconomic data
macro_data = pd.read_csv('macro_features.csv')
macro_columns = macro_data.columns

# Factors with lags are not used in the model. Remove factors with lags
new_macro_columns = [col for col in macro_columns if '_lag' not in col]
macro_data = macro_data[new_macro_columns]

### Merging bank panel and macroeconomic time series

In [7]:
# Merge the bank panel and macroeconomic indicators
result_df = merge_bank_macro_datasets(bank_data, macro_data) #, pca_data, macro_data1)

In [8]:
# Delete Nans values due to the lag of the response variable
result_df.dropna(subset=['Provision_Lag1'], inplace=True)
result_df.reset_index(drop=True, inplace=True)

In [9]:
result_df.head(5)

,Report Date,IDRSSD,Financial Institution Name,Provision for Loan Lease Losses as % of Aver. Assets,Real GDP growth,Nominal GDP growth,Unemployment rate,Unemployment rate change,3-month Treasury rate change,BBB corporate yield,BBB corporate yield change,Dow Jones Total Stock Market Index change,Commercial Real Estate Price Index change,Market Volatility Index,Market Volatility Index change,Real GDP growth_ema3,Nominal GDP growth_ema1.75,Market Volatility Index_ema4,Market Volatility Index change_ema10,Provision_Lag1
0,2003-03-31,12311,"HUNTINGTON NATIONAL BANK, THE",0.54,2.1,4.1,5.9,0.2,-0.5,6.2,-1.2,-0.252784,0.083916,34.7,0.329502,1.601115,3.577538,35.9816,0.160864,0.74
1,2003-03-31,14409,CITIZENS BANK OF MASSACHUSETTS,0.32,2.1,4.1,5.9,0.2,-0.5,6.2,-1.2,-0.252784,0.083916,34.7,0.329502,1.601115,3.577538,35.9816,0.160864,0.30
2,2003-03-31,17147,"FIRST MERCHANTS BANK, NATIONAL ASSOCIATION",1.77,2.1,4.1,5.9,0.2,-0.5,6.2,-1.2,-0.252784,0.083916,34.7,0.329502,1.601115,3.577538,35.9816,0.160864,0.31
3,2003-03-31,23504,"BRIDGEHAMPTON NATIONAL BANK, THE",0.00,2.1,4.1,5.9,0.2,-0.5,6.2,-1.2,-0.252784,0.083916,34.7,0.329502,1.601115,3.577538,35.9816,0.160864,0.05
4,2003-03-31,30810,DISCOVER BANK,5.93,2.1,4.1,5.9,0.2,-0.5,6.2,-1.2,-0.252784,0.083916,34.7,0.329502,1.601115,3.577538,35.9816,0.160864,6.20


### Additional features

In [10]:
# Adding Fixed Effects
# Assuming 'Financial Institution Name' is a categorical variable, so we can one-hot encode it
result_df = pd.get_dummies(result_df, columns=['IDRSSD'], drop_first=True)

## Supervised model

In [11]:
# Response variable column
y_col = 'Provision for Loan Lease Losses as % of Aver. Assets'

In [12]:
scaler = StandardScaler()
gb_model = GradientBoostingRegressor(random_state=42)

pipeline = Pipeline(steps=[("scaler", scaler), ("gb_model", gb_model)])

### Train-test split

In [13]:
# The last year is for test,remaining data - for train
data_set_train = result_df[result_df['Report Date']<='2021-12-31'].copy()
data_set_test = result_df[result_df['Report Date']>'2021-12-31'].copy()

#### Removing outliers from the train set

In [14]:
lower_limit = np.percentile(data_set_train[y_col], 0.5)
upper_limit = np.percentile(data_set_train[y_col], 99)

data_set_train = data_set_train[(data_set_train[y_col]<=upper_limit)&(data_set_train[y_col]>=lower_limit)].copy()
data_set_train.reset_index(drop=True, inplace=True)

## Optimizing Model Selection: Factor Selection and Hyperparameter Tuning
The analysis on time series data for the "average" bank indicates that the response variable is most influenced by its lag of one quarter. Moreover, the residuals of the autoregressive time series model AR(1) exhibit the strongest correlation with the following factors: 'Real GDP growth_ema3', 'BBB corporate yield', '3-month Treasury rate change', 'Dow Jones Total Stock Market Index change', 'Market Volatility Index', 'Market Volatility Index change'. 

To select the optimal model, I created a list of model variants that include these influential factors (models1-models11). Addidtionally, I added model13 based on the results of usupervised learning.

Model selection and hyperparameter tuning are performed using cross-validation, allowing us to determine the best model and optimal hyperparameter values.

In [15]:
# Custom cross-validation split for panel data
panel_cv = PanelDataSplit(test_size=4, date_axis=data_set_train['Report Date'], n_splits=5)

In [16]:
# Models
models = {'model1': ['Provision_Lag1', 'Real GDP growth_ema3', 'BBB corporate yield'],
          'model2': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index'],
          'model3': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change'],
          'model4': ['Provision_Lag1', 'Real GDP growth_ema3', 'Dow Jones Total Stock Market Index change'],
          'model5': ['Provision_Lag1', 'Real GDP growth_ema3', '3-month Treasury rate change'],
          'model6': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index', 'BBB corporate yield'],
          'model7': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index', '3-month Treasury rate change'],
          'model8': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change', 'BBB corporate yield'],
          'model9': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change', '3-month Treasury rate change'],
          'model10': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change', '3-month Treasury rate change', 'BBB corporate yield'],
          'model11': ['Provision_Lag1', 'Real GDP growth_ema3', 'BBB corporate yield', '3-month Treasury rate change'],
         # 'model12': ['Provision_Lag1'] + list(pca_data.columns[:-1]), #PCA components
          'model13': ['Provision_Lag1', 'Japan bilateral dollar exchange rate (yen/USD)', 
                      'Euro area bilateral dollar exchange rate (USD/euro)',
                      'NBER_Recession_Indicator_Peak_through_Trough', 'Commercial_Banks_Treasury_and_Agency_Securities',
                      'Real disposable income growth', 'U.K. bilateral dollar exchange rate (USD/pound)', 
                      'Unemployment rate', 'BBB corporate yield', 'Households_Net_Worth', 'Euro area inflation', 
                      'Market Volatility Index', 'Developing Asia inflation'],
                             
         }

In [17]:
# Grid to search over to be able get better test results and reduce overfitting

# Define the parameters
param_grid = {
    'gb_model__n_estimators': [100, 200, 300, 400, 500],
    'gb_model__learning_rate': [0.05, 0.1, 0.2, 0.3],
    'gb_model__max_depth': [3, 4, 5]
}


# Initialize the grid search
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='r2', cv=panel_cv)

In [ ]:
best_model_name, best_score, best_model, models_results, estimators = \
    search_best_model(data_set_train, models, grid_search, 
                      y_col, [col for col in result_df.columns if col.startswith('IDRSSD_')])

### Results of model selection

In [ ]:
best_model_name

In [ ]:
# Factors of the best model
models[best_model_name]

In [ ]:
# Pipeline for the best model
best_model

In [ ]:
models_results['Cross-Validation R^2 Standard Error of the Mean'] = \
    models_results['Cross-Validation R^2 std'] / panel_cv.get_n_splits()**0.5

models_results

In [ ]:
models_results.to_csv('GB_models_results.csv')

## Final model after hyperparameter tuning: Test sample performance

In [37]:
# Best model features
features_all = models[best_model_name] + [col for col in result_df.columns if col.startswith('IDRSSD_')]

X_train = data_set_train[features_all]
X_test = data_set_test[features_all]

In [43]:
# Fit the best model using the whole train set
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_train)

### Train sample performance

In [44]:
estimate_errors(y_train, y_pred, lower_limit, upper_limit)

,measure
R squared,0.882779
RMSE,0.297373
"median relative error, %",17.976709


### Test sample performance

In [45]:
y_pred = best_model.predict(X_test)

In [46]:
# All errors together (excluding outliers)
estimate_errors(y_test, y_pred, lower_limit, upper_limit)

,measure
R squared,0.821973
RMSE,0.221627
"median relative error, %",40.290357
